# Monitoring setup for Bicycle Sharing Demand Prediction

This notebook shows how you can use the Evidently and MLflow to:
* calculate prerformance and data drift for the model, performed as batch checks 
* log modeles quality & data drift using MLflow Tracking
* explore the result using MLflow UI

More examples are avaliable in the github: https://github.com/evidentlyai/evidently/tree/main/examples

Evidently dosc: https://docs.evidentlyai.com/

Join our Discord: https://discord.com/invite/xZjKRaNp8b

In [1]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import DataDriftTab, NumTargetDriftTab, RegressionPerformanceTab
from evidently.options import DataDriftOptions
from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection, RegressionPerformanceProfileSection

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [3]:
content = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday']) 

In [ ]:
raw_data.head()

In [4]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [ ]:
raw_data.head()

## Model training 

In [5]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]#'weathersit']

In [6]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    reference[numerical_features + categorical_features],
    reference[target],
    test_size=0.3
)

In [8]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [9]:
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=0)

In [10]:
preds_train = regressor.predict(X_train)
preds_test = regressor.predict(X_test)

## Model validation

In [11]:
X_train['target'] = y_train
X_train['prediction'] = preds_train

X_test['target'] = y_test
X_test['prediction'] = preds_test

In [12]:
column_mapping = ColumnMapping()

column_mapping.target = 'target'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [13]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab()])
regression_perfomance_dashboard.calculate(X_train.sort_index(), X_test.sort_index(), 
                                          column_mapping=column_mapping)

In [14]:
regression_perfomance_dashboard.show()

## Production model training

In [15]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [16]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [17]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
reference['prediction'] = ref_prediction

In [18]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab(verbose_level=0)])
regression_perfomance_dashboard.calculate(reference, None, column_mapping=column_mapping)
regression_perfomance_dashboard.show()

## Some time has passed - how is the model working?

In [19]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])
current['prediction'] = current_prediction

### Week 1

In [20]:
regression_perfomance_dashboard.calculate(reference, current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                            column_mapping=column_mapping)
regression_perfomance_dashboard.show()

### Week 2

In [21]:
RegressionPerformanceTab.list_widgets()

['Regression Model Performance Report.',
 'Reference: Model Quality (+/- std)',
 'Current: Model Quality (+/- std)',
 'Reference: Predicted vs Actual',
 'Current: Predicted vs Actual',
 'Reference: Predicted vs Actual in Time',
 'Current: Predicted vs Actual in Time',
 'Reference: Error (Predicted - Actual)',
 'Current: Error (Predicted - Actual)',
 'Reference: Absolute Percentage Error',
 'Current: Absolute Percentage Error',
 'Reference: Error Distribution',
 'Current: Error Distribution',
 'Reference: Error Normality',
 'Current: Error Normality',
 'Reference: Mean Error per Group (+/- std)',
 'Current: Mean Error per Group (+/- std)',
 'Reference: Predicted vs Actual per Group',
 'Current: Predicted vs Actual per Group',
 'Error Bias: Mean/Most Common Feature Value per Group']

In [22]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab(include_widgets=[
    'Regression Model Performance Report.',
    'Reference: Model Quality (+/- std)',
    'Current: Model Quality (+/- std)',
    'Current: Error (Predicted - Actual)',
    'Current: Error Distribution',
])])
regression_perfomance_dashboard.calculate(reference, current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                                            column_mapping=column_mapping)
regression_perfomance_dashboard.show()

### Week 3

In [23]:
regression_perfomance_dashboard.calculate(reference, current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                                            column_mapping=column_mapping)
regression_perfomance_dashboard.show()

## Why the quality has dropped?

In [24]:
column_mapping_drift = ColumnMapping()

column_mapping_drift.target = target
column_mapping_drift.prediction = prediction
column_mapping_drift.numerical_features = numerical_features
column_mapping_drift.categorical_features = []

In [25]:
data_drift_dashboard = Dashboard(tabs=[DataDriftTab()])
data_drift_dashboard.calculate(reference,
                               current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'], 
                               column_mapping_drift
                              )
data_drift_dashboard.show()

## Let's customize the report!

In [28]:
def anderson_stat_test(reference_data: pd.DataFrame, current_data: pd.DataFrame):
    return anderson_ksamp(np.array([reference_data, current_data]))[2]

options = DataDriftOptions(feature_stattest_func=anderson_stat_test, nbinsx=20, confidence=0.90)

In [29]:
the_dashboard = Dashboard(
    tabs=[RegressionPerformanceTab(include_widgets=[
                                    'Regression Model Performance Report.',
                                    'Reference: Model Quality (+/- std)',
                                    'Current: Model Quality (+/- std)',
                                    'Current: Error (Predicted - Actual)',
                                    'Current: Error Distribution',]
                                  ),
          DataDriftTab()],
    options=[options])
                                
the_dashboard.calculate(reference,
                        current.loc['2011-02-14 00:00:00':'2011-02-21 23:00:00'], 
                        column_mapping_drift)

the_dashboard.show()

## Jupyter vs Automated Run?

To run this part of the tutorial you need to install MLflow (if not installed yet)

You can install MLflow with the following command: `pip install mlflow` or install MLflow with scikit-learn via `pip install mlflow[extras]`

More details:https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#id5

In [30]:
import mlflow
#import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [31]:
experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),  
]

In [32]:
model_profile = Profile(sections=[DataDriftProfileSection(), RegressionPerformanceProfileSection()])
model_profile.calculate(reference, 
                        current.loc[experiment_batches[0][0]:experiment_batches[0][1]],
                        column_mapping=column_mapping_drift)

In [33]:
logged_json_profile = json.loads(model_profile.json())

In [36]:
logged_json_profile["regression_performance"]["data"]["metrics"]["current"]["mean_error"]

-6.26536170212766

In [37]:
logged_json_profile["data_drift"]["data"]["metrics"]["share_drifted_features"]

0.7142857142857143

In [38]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_experiment('Model Quality Evaluation with Evidently')

#generate model profile
model_profile = Profile(sections=[DataDriftProfileSection(), RegressionPerformanceProfileSection()])

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: #inside brackets run_name='test'
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Get metrics
        model_profile.calculate(reference, 
                        current.loc[date[0]:date[1]],
                        column_mapping=column_mapping_drift)
        logged_json_profile = json.loads(model_profile.json())
        
        me = logged_json_profile["regression_performance"]["data"]["metrics"]["current"]["mean_error"]
        mae = logged_json_profile["regression_performance"]["data"]["metrics"]["current"]["mean_abs_error"]
        drift_share = logged_json_profile["data_drift"]["data"]["metrics"]["share_drifted_features"]
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        mlflow.log_metric('drift_share', round(drift_share, 3))

        print(run.info)

<RunInfo: artifact_uri='file:///Users/emeli/Dev/evidently_dev/examples/data_stories/mlruns/1/a6cb95789f014046b4edb80948678679/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='a6cb95789f014046b4edb80948678679', run_uuid='a6cb95789f014046b4edb80948678679', start_time=1644849316384, status='RUNNING', user_id='emeli'>
<RunInfo: artifact_uri='file:///Users/emeli/Dev/evidently_dev/examples/data_stories/mlruns/1/ddd911dc32bc4de8884851386441eada/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='ddd911dc32bc4de8884851386441eada', run_uuid='ddd911dc32bc4de8884851386441eada', start_time=1644849317730, status='RUNNING', user_id='emeli'>
<RunInfo: artifact_uri='file:///Users/emeli/Dev/evidently_dev/examples/data_stories/mlruns/1/eebfdfaaaa7046fe8f2d096b33f4fa81/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='eebfdfaaaa7046fe8f2d096b33f4fa81', run_uuid='eebfdfaaaa7046fe8f2d096b33f4fa81', start_time=164484931

In [ ]:
#run MLflow UI (NOT recommented! It will be more convinient to run it directly from the TERMINAL)
#!mlflow ui